In [1]:
import pandas as pd
import csv
import numpy as np
import time

from Helper.Preprocessing import *
from Helper.Model import *
from Helper.GeneticAlgorithm import GeneticAlgorithm
from Helper.PSO import PSO
from Helper.FileManager import FileManager

In [2]:
dfx = pd.read_csv("../ufjf_dataset.csv")
dfx['time1'] = pd.to_datetime(dfx['time1'])
dfx.index = dfx['time1']
dfx['radio_id'] = dfx['radio_id'].str.lower()
dfx.head(10)

,time1,timet,client_id,radio_id
time1,,,,
2019-05-31 08:48:52,2019-05-31 08:48:52,13.0,7,economia-4andar
2019-05-31 08:48:52,2019-05-31 08:48:52,31.0,20,ru_nutricionistas
2019-05-31 08:48:53,2019-05-31 08:48:53,38.0,29,ap4600-67aef4
2019-05-31 08:48:53,2019-05-31 08:48:53,17.0,12,reitoria-centralatendimento
2019-05-31 08:48:54,2019-05-31 08:48:54,10.0,5,ccs-2andarfundos
2019-05-31 08:48:55,2019-05-31 08:48:55,9.0,6,engenharia-labspot
2019-05-31 08:48:56,2019-05-31 08:48:56,1.0,1,predioitamarfranco3andarsala5306
2019-05-31 08:48:56,2019-05-31 08:48:56,2.0,2,sala4157ladocantinaengenharia
2019-05-31 08:48:56,2019-05-31 08:48:56,2.0,3,iad-1andar


In [3]:
dfxx = dfx[dfx['radio_id'].str.contains('ru')]
l_ru = list(dfxx['radio_id'].unique())
l_ru.remove('deptconstrucaocivilsala4107')
l_ru

df = dfx[dfx['radio_id'].str.contains('|'.join(l_ru))]

X, Y = Preprocessing(df).get_data()

Pre-processing...


# Common Variables

In [4]:
generation = 20
population_size = 20
P = 11
Q = 3

# MLP Hyperparameters

In [ ]:
n_MAX = 200
n_MIN = 20
learning_rate_MAX = 0.0009
learning_rate_MIN = 0.00001
maxiter = 2000

boundaries = np.zeros([2,2])
boundaries[0,0] = n_MIN
boundaries[1,0] = n_MAX
boundaries[0,1] = learning_rate_MIN
boundaries[1,1] = learning_rate_MAX

boundaries_type = [None] * 2
boundaries_type[0] = True # Integer
boundaries_type[1] = False # Float

print(boundaries[0,:])

# Genetic Algorithm - MLP

In [ ]:
ga = GeneticAlgorithm(population_size, boundaries, boundaries_type)
fm = FileManager()
fm.create_file("ga_test")
index = 0
while index <= generation:
    index += 1
    print("\n--- Generation {} ---".format(index))
    ga.generate()  
    population = ga.get_population()
    for i in population:
        start_time = time.time()
        parameters = i.get_parameters()
        
        model = Model(X, Y, model_type = 'MLP', n_neurons = parameters[0], learning_rate = parameters[1])
        
        fitness_r2 = model.fit_predict_evaluate(n_splits = 3, n_repeats = 5)
        
        ga.evaluate(i, fitness_r2)
        ga_time = time.time() - start_time
        fm.write2file(parameters, P, Q, fitness_r2, ga_time)
    
    ga.print_population()
    ga.print_best_solution()
fm.close_file()

# PSO - MLP

In [ ]:
# Initializing the variables and the population
pso = PSO(population_size, boundaries, boundaries_type)
fm = FileManager()
pop = pso.get_population()
fm.create_file("pso_test")

print('\nInitializing the population...')
for p in pop:
    start_time = time.time()
    param = p.get_position()

    model = Model(X, Y, model_type = 'MLP', n_neurons = int(param[0]), learning_rate = param[1])
    fitness_r2 = model.fit_predict_evaluate(n_splits = 3, n_repeats = 5)

    pso.insert_particle_fitness(p, fitness_r2)
    pso_time = time.time() - start_time
    fm.write2file(param, P, Q, fitness_r2, pso_time)

pso.print_global_best_particle()

iteration = 0
print('\nRunning PSO Loop...')
while(iteration <= generation):
    iteration += 1
    print('\nRunning... : {} of {}.'.format(iteration, generation))
    for p in pop:
        start_time = time.time()
        print("Particle {}.".format(p.get_index()))
        pso.calculate_position_velocity(p)
        param = p.get_position()
        print(param)

        model = Model(X, Y, model_type = 'MLP', n_neurons = int(param[0]), learning_rate = param[1])
        fitness_r2 = model.fit_predict_evaluate(n_splits = 3, n_repeats = 5)
        
        pso.insert_particle_fitness(p, fitness_r2)
        pso_time = time.time() - start_time
        fm.write2file(param, P, Q, fitness_r2, pso_time)
        
    pso.print_global_best_particle()
    print("GBest_swap = {}".format(pso.get_best_particle_swap()))
    
fm.close_file()

# DT Hyperparameters

In [8]:
max_depth_MAX = 200
max_depth_MIN = 10
min_samples_split_MAX = 22
min_samples_split_MIN = 2

boundaries = np.zeros([2,2])
boundaries[0,0] = max_depth_MIN
boundaries[1,0] = max_depth_MAX
boundaries[0,1] = min_samples_split_MIN
boundaries[1,1] = min_samples_split_MAX

boundaries_type = [None] * 2
boundaries_type[0] = True # Integer
boundaries_type[1] = True # Integer

print(boundaries[0,:])

[10.  2.]


# Genetic Algorithm - DT

In [6]:
ga = GeneticAlgorithm(population_size, boundaries, boundaries_type)
fm = FileManager()
fm.create_file("ga_test_dt")
index = 0
while index <= generation:
    index += 1
    print("\n--- Generation {} ---".format(index))
    ga.generate()  
    population = ga.get_population()
    for i in population:
        start_time = time.time()
        parameters = i.get_parameters()
        
        model = Model(X, Y, model_type = 'DT', max_depth = parameters[0], min_samples_split = parameters[1])
        
        fitness_r2 = model.fit_predict_evaluate(n_splits = 3, n_repeats = 5)
        
        ga.evaluate(i, fitness_r2)
        ga_time = time.time() - start_time
        fm.write2file(parameters, P, Q, fitness_r2, ga_time)
    
    ga.print_population()
    ga.print_best_solution()
fm.close_file()


--- Generation 1 ---
Individual 1: Parameters:(95,16) Fitness: 0.9021782807381804
Individual 2: Parameters:(113,12) Fitness: 0.8980497670418777
Individual 3: Parameters:(189,7) Fitness: 0.8899248242912848
Individual 4: Parameters:(175,12) Fitness: 0.8970750375367533
Individual 5: Parameters:(70,8) Fitness: 0.8926912278582833
Individual 6: Parameters:(189,10) Fitness: 0.8957873168494221
Individual 7: Parameters:(198,10) Fitness: 0.8954151428338203
Individual 8: Parameters:(17,12) Fitness: 0.898952977230082
Individual 9: Parameters:(17,12) Fitness: 0.8985525861086195
Individual 10: Parameters:(151,3) Fitness: 0.8824125809206975
Individual 11: Parameters:(174,5) Fitness: 0.8862163026535423
Individual 12: Parameters:(113,7) Fitness: 0.890271892814498
Individual 13: Parameters:(198,9) Fitness: 0.893541534397375
Individual 14: Parameters:(151,7) Fitness: 0.88824424286161
Individual 15: Parameters:(32,9) Fitness: 0.894344924660208
Individual 16: Parameters:(175,12) Fitness: 0.897500387864754

Individual 1: Parameters:(17,12) Fitness: 0.9003983101234333
Individual 2: Parameters:(61,12) Fitness: 0.8983513962959754
Individual 3: Parameters:(78,7) Fitness: 0.8897985259681732
Individual 4: Parameters:(17,5) Fitness: 0.8896812201109081
Individual 5: Parameters:(70,7) Fitness: 0.8880125287997237
Individual 6: Parameters:(17,5) Fitness: 0.8881374579631679
Individual 7: Parameters:(17,12) Fitness: 0.8978261042850341
Individual 8: Parameters:(71,5) Fitness: 0.8864573473391493
Individual 9: Parameters:(20,7) Fitness: 0.8899649085771822
Individual 10: Parameters:(17,5) Fitness: 0.8879855535890859
Individual 11: Parameters:(71,12) Fitness: 0.8986997843065495
Individual 12: Parameters:(182,5) Fitness: 0.8882747884427895
Individual 13: Parameters:(17,5) Fitness: 0.8879311147961427
Individual 14: Parameters:(17,12) Fitness: 0.8975362040980954
Individual 15: Parameters:(70,8) Fitness: 0.8913897298037284
Individual 16: Parameters:(61,7) Fitness: 0.8917944297765176
Individual 17: Parameters:(

Individual 1: Parameters:(17,8) Fitness: 0.8943095858311854
Individual 2: Parameters:(70,11) Fitness: 0.8965792092450037
Individual 3: Parameters:(167,2) Fitness: 0.8811441609997545
Individual 4: Parameters:(17,11) Fitness: 0.8982271273837548
Individual 5: Parameters:(105,8) Fitness: 0.894193527018351
Individual 6: Parameters:(17,5) Fitness: 0.8876283326100743
Individual 7: Parameters:(17,8) Fitness: 0.8921288212607081
Individual 8: Parameters:(101,11) Fitness: 0.898353175756737
Individual 9: Parameters:(164,8) Fitness: 0.8911042948638066
Individual 10: Parameters:(70,7) Fitness: 0.8924929589657021
Individual 11: Parameters:(105,2) Fitness: 0.8805906756787683
Individual 12: Parameters:(67,13) Fitness: 0.899907923404207
Individual 13: Parameters:(70,13) Fitness: 0.8988531749601705
Individual 14: Parameters:(17,5) Fitness: 0.8879970318721074
Individual 15: Parameters:(70,13) Fitness: 0.8989740057095292
Individual 16: Parameters:(101,2) Fitness: 0.8812951064719813
Individual 17: Parameter

# PSO - DT

In [10]:
# Initializing the variables and the population
pso = PSO(population_size, boundaries, boundaries_type)
fm = FileManager()
pop = pso.get_population()
fm.create_file("pso_test_dt")

print('\nInitializing the population...')
for p in pop:
    start_time = time.time()
    param = p.get_position()

    model = Model(X, Y, model_type = 'DT', max_depth = int(param[0]), min_samples_split = int(param[1]))
    fitness_r2 = model.fit_predict_evaluate(n_splits = 3, n_repeats = 5)

    pso.insert_particle_fitness(p, fitness_r2)
    pso_time = time.time() - start_time
    fm.write2file(param, P, Q, fitness_r2, pso_time)

pso.print_global_best_particle()

iteration = 0
print('\nRunning PSO Loop...')
while(iteration <= generation):
    iteration += 1
    print('\nRunning... : {} of {}.'.format(iteration, generation))
    for p in pop:
        start_time = time.time()
        print("Particle {}.".format(p.get_index()))
        pso.calculate_position_velocity(p)
        param = p.get_position()
        print(param)

        model = Model(X, Y, model_type = 'DT', max_depth = int(param[0]), min_samples_split = int(param[1]))
        fitness_r2 = model.fit_predict_evaluate(n_splits = 3, n_repeats = 5)
        
        pso.insert_particle_fitness(p, fitness_r2)
        pso_time = time.time() - start_time
        fm.write2file(param, P, Q, fitness_r2, pso_time)
        
    pso.print_global_best_particle()
    print("GBest_swap = {}".format(pso.get_best_particle_swap()))
    
fm.close_file()


Initializing the population...
-----------------------------------------------
Best Particle: 15
Best Position:[154, 19]
Best Fitness: 0.9075202263775249
-----------------------------------------------

Running PSO Loop...

Running... : 1 of 20.
Particle 0.
[165, 6]
Particle 1.
[115, 17]
Particle 2.
[154, 14]
Particle 3.
[176, 19]
Particle 4.
[83, 4]
Particle 5.
[107, 14]
Particle 6.
[154, 8]
Particle 7.
[153, 11]
Particle 8.
[101, 19]
Particle 9.
[31, 6]
Particle 10.
[53, 13]
Particle 11.
[172, 14]
Particle 12.
[156, 17]
Particle 13.
[68, 12]
Particle 14.
[71, 9]
Particle 15.
[149, 19]
Particle 16.
[42, 7]
Particle 17.
[132, 12]
Particle 18.
[152, 6]
Particle 19.
[116, 8]
-----------------------------------------------
Best Particle: 15
Best Position:[154, 19]
Best Fitness: 0.9075202263775249
-----------------------------------------------
GBest_swap = 3

Running... : 2 of 20.
Particle 0.
[146, 7]
Particle 1.
[134, 19]
Particle 2.
[164, 16]
Particle 3.
[194, 19]
Particle 4.
[102, 6]


Particle 1.
[169, 22]
Particle 2.
[174, 22]
Particle 3.
[171, 22]
Particle 4.
[174, 22]
Particle 5.
[161, 22]
Particle 6.
[182, 22]
Particle 7.
[170, 22]
Particle 8.
[175, 22]
Particle 9.
[173, 22]
Particle 10.
[184, 22]
Particle 11.
[186, 22]
Particle 12.
[176, 22]
Particle 13.
[169, 22]
Particle 14.
[183, 22]
Particle 15.
[167, 22]
Particle 16.
[169, 22]
Particle 17.
[163, 22]
Particle 18.
[162, 22]
Particle 19.
[169, 22]
-----------------------------------------------
Best Particle: 13
Best Position:[169, 22]
Best Fitness: 0.9113877372573954
-----------------------------------------------
GBest_swap = 16

Running... : 14 of 20.
Particle 0.
[168, 21]
Particle 1.
[163, 22]
Particle 2.
[171, 22]
Particle 3.
[170, 22]
Particle 4.
[177, 22]
Particle 5.
[166, 22]
Particle 6.
[165, 22]
Particle 7.
[174, 22]
Particle 8.
[177, 22]
Particle 9.
[171, 22]
Particle 10.
[165, 22]
Particle 11.
[179, 22]
Particle 12.
[163, 22]
Particle 13.
[159, 22]
Particle 14.
[164, 22]
Particle 15.
[173, 22]
Par